In [ ]:
import pandas as pd
import numpy as np
import google.colab
google.colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/hackathon_data/avk_hackathon_data_transactions.csv')
df = df.drop_duplicates(subset =["merchant_type"]) 

In [ ]:
df = df.sort_values(by=['category']).dropna()
unic_cats = df.category.unique()
unic_cats

array(['Авиабилеты', 'Аренда авто', 'Госсборы', 'Дом/Ремонт',
       'Ж/д билеты', 'Животные', 'Кино', 'Книги', 'Красота', 'Музыка',
       'НКО', 'Образование', 'Одежда/Обувь', 'Отели', 'Развлечения',
       'Разные товары', 'Связь/Телеком', 'Сервисные услуги',
       'Спорттовары', 'Супермаркеты', 'Транспорт', 'Фаст Фуд',
       'Финансовые услуги', 'Фото/Видео'], dtype=object)

In [ ]:
DROP_CAT = ['НКО', 'Наличные', 'Супермаркеты', 'Финансовые услуги']
DROP_MERCH = []
for cat in DROP_CAT:
    L  = list(df[df['category'] == cat]['merchant_type'].values)
    for l in L:
        DROP_MERCH.append(l)
DROP_MERCH

[484.0, 491.0, 286.0, 336.0, 385.0, 383.0, 388.0, 391.0]

In [ ]:
CAT_TYPES = pd.read_csv('/content/drive/My Drive/hackathon_data/CAT_TYPES.csv').drop(columns = ['Unnamed: 0'])
CAT_TYPES

,Duty Free,Авиабилеты,Автоуслуги,Аптеки,Аренда авто,Госсборы,Дом/Ремонт,Ж/д билеты,Животные,Искусство,Кино,Книги,Красота,Медицинские услуги,Музыка,НКО,Наличные,Образование,Одежда/Обувь,Отели,Развлечения,Разные товары,Рестораны,Связь/Телеком,Сервисные услуги,Спорттовары,Сувениры,Супермаркеты,Топливо,Транспорт,Турагентства,Фаст Фуд,Финансовые услуги,Фото/Видео,Цветы,Частные услуги
0,281.0,55.0,436.0,264.0,429.0,492.0,320.0,227.0,374.0,340.0,448.0,270.0,410.0,474.0,326.0,484.0,379.0,478.0,350.0,170.0,452.0,375.0,327.0,243.0,355.0,342.0,348.0,288.0,298.0,279.0,239.0,330.0,385.0,415.0,371.0,406.0
1,NaN,97.0,293.0,335.0,102.0,495.0,441.0,225.0,NaN,362.0,446.0,26.0,367.0,469.0,324.0,483.0,380.0,481.0,307.0,212.0,449.0,262.0,328.0,422.0,357.0,310.0,NaN,291.0,269.0,235.0,234.0,NaN,383.0,414.0,271.0,405.0
2,NaN,81.0,294.0,NaN,107.0,498.0,251.0,NaN,NaN,363.0,447.0,263.0,409.0,465.0,NaN,486.0,NaN,480.0,305.0,180.0,453.0,349.0,329.0,420.0,417.0,341.0,NaN,289.0,297.0,226.0,NaN,NaN,388.0,426.0,NaN,411.0
3,NaN,94.0,292.0,NaN,430.0,497.0,18.0,NaN,NaN,361.0,NaN,343.0,401.0,255.0,NaN,490.0,NaN,479.0,315.0,150.0,461.0,407.0,NaN,247.0,412.0,NaN,NaN,287.0,369.0,236.0,NaN,NaN,391.0,253.0,NaN,403.0
4,NaN,41.0,301.0,NaN,109.0,493.0,276.0,NaN,NaN,364.0,NaN,373.0,NaN,471.0,NaN,489.0,NaN,477.0,314.0,116.0,450.0,283.0,NaN,242.0,424.0,NaN,NaN,286.0,NaN,302.0,NaN,NaN,390.0,252.0,NaN,404.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_merch_category(merh_type):
  
    for index, row in CAT_TYPES.iterrows():
        for i in range(len(CAT_TYPES.columns)): 
            if row[i] == merh_type:
                return row.index[i]

    return 'Пошел НАХУЙ'

def get_merch_types(merh_category):
  
    c = CAT_TYPES[merh_category].values
    c = c[~np.isnan(c)]
    return c

get_merch_category(1.0)

'Пошел НАХУЙ'

In [ ]:
data = pd.read_csv('/content/drive/My Drive/hackathon_data/UIM_train.csv').drop(columns = ['Unnamed: 0']).dropna()
data

,party_rk,transaction_amt_rur,286,1,291,330,335,5,328,297,381,336,380,284,378,388,313,309,274,367,283,226,285,275,229,296,323,342,311,491,346,447,371,322,242,343,474,374,230,401,...,223,428,430,438,184,165,183,182,88,87,120,80,78,72,70,68,132,417,61,57,152,153,48,159,163,170,172,173,96,180,181,444,gender_cd,age,marital_status_desc,children_cnt,region_flg,user_transactions_by_month,n_prods,user_transactions_cat
0,1,108547.37,0.104693,0.548000,0.000000,0.020811,0.000000,0.000000,0.000000,0.028461,0.000000,0.008006,0.000000,0.000000,0.000000,0.213732,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,55.0,2,1,0,16502.124545,2,1.0
1,5,838288.67,0.120558,0.007862,0.000000,0.006429,0.004902,0.000000,0.021385,0.016726,0.506701,0.001416,0.000000,0.037624,0.000000,0.000000,0.002515,0.000000,0.003509,0.000000,0.003974,0.0,0.001941,0.017296,0.000260,0.000000,0.048932,0.0,0.000000,0.0,0.000000,0.0,0.007396,0.009070,0.000000,0.0,0.004156,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,45.0,2,1,0,110706.375833,2,5.0
2,7,496285.94,0.000000,0.494378,0.000000,0.000000,0.003715,0.000000,0.000000,0.010577,0.000000,0.000000,0.482585,0.000000,0.000000,0.000000,0.008745,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,45.0,2,4,0,376685.954444,1,5.0
3,8,131087.00,0.002472,0.883619,0.000000,0.011298,0.004959,0.007629,0.007239,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004013,0.000526,0.030018,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.044497,0.0,0.002372,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,60.0,2,1,0,19532.428571,2,1.0
4,9,30508.05,0.096337,0.590008,0.008260,0.026583,0.000000,0.017536,0.000000,0.000000,0.003606,0.002917,0.000000,0.000000,0.000000,0.153927,0.000000,0.000000,0.064311,0.000000,0.000000,0.0,0.000000,0.000000,0.034614,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.001246,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,40.0,2,1,0,4319.370000,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45191,94602,82888.93,0.312945,0.008795,0.003302,0.010663,0.002328,0.003836,0.000000,0.047341,0.148367,0.000000,0.000000,0.000000,0.011811,0.395433,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.014055,0.000000,0.026831,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,35.0,2,0,0,12019.366667,3,1.0
45192,94603,49377.23,0.032834,0.037284,0.000000,0.002248,0.000000,0.000000,0.000000,0.000000,0.629966,0.000000,0.206512,0.081556,0.003524,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.006076,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,

In [ ]:
from sklearn.neighbors import NearestNeighbors

''' FIT Classifaer'''

data_np = data.drop(columns=['party_rk']).to_numpy()

neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(data_np)

example = data_np[6666]
print(neigh.kneighbors([example]))

(array([[  0.        ,  74.65368402, 182.46551108, 258.40206156,
        270.48652451, 306.2464868 , 327.31633914, 360.0406807 ,
        368.90691475, 463.89265151]]), array([[ 6666, 27156, 10425,  2101, 29556, 33888, 29845, 24344,  9743,
        22683]]))


In [ ]:
user = 310
last_merchant = 286.0
last_merchant_type = 'merchant_type_286'

def make_advice(user, merhant_type):
      dists, top_neighs = neigh.kneighbors([data_np[user]])

      top_neighs = top_neighs[:,1:].reshape(-1)
      dists = dists[:,1:].reshape(-1)

      #sum_interactions = np.sum([data.loc[top_neighs[n]][2:-9].values/dists[n] for n in range(len(top_neighs))], axis=0)
      top_neighs = [data.loc[n][0] for n in top_neighs]
      sum_interactions = np.sum([data.loc[data['party_rk'] == top_neighs[n]].values.reshape(-1)[2:-9]/dists[n] for n in range(len(top_neighs))], axis=0)

      user_merches = data.loc[user][2:-9]
      user_merches = user_merches[user_merches > 0].index

      sum_interactions = sum_interactions / sum_interactions.sum()
      relevant_merhces_idx = sum_interactions.argsort()[::-1]
      relevant_merches = [data.columns[i+2] for i in relevant_merhces_idx]

      new_merches = [item for item in relevant_merches if item not in user_merches if item not in DROP_MERCH]

      merch_C = get_merch_category(last_merchant)
      similiar_merhces = get_merch_types(merch_C)

      new_category_merches = [item for item in new_merches if item in similiar_merhces]

      if len(new_category_merches) != 0:
            top_advice = new_category_merches[0]

      elif len(new_merches) != 0:
            top_advice = new_merches[0]

      else:
            top_advice = relevant_merches[0]

      idx = relevant_merches.index(top_advice)
      idx = np.where(relevant_merhces_idx == idx)

      sum_interactions[idx[0][0]]

      return top_advice

advice = make_advice(user, last_merchant)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [ ]:
print('Hey yo, man, you just visited', last_merchant_type)
print('We recommend you also to visit', 'merchant_type_'+str(advice))

Hey yo, man, you just visited merchant_type_286
We recommend you also to visit merchant_type_388


In [ ]:
import pickle

DROP_CAT = np.array(['НКО', 'Наличные', 'Супермаркеты', 'Финансовые услуги'])

preds = pickle.load(open(f"/content/drive/My Drive/hackathon_data/preds (3).pkl", 'rb'))
parties = pickle.load(open(f"/content/drive/My Drive/hackathon_data/parties.pkl", 'rb'))

In [ ]:
pred_MERCH_type = [p for p in preds[22] if p not in DROP_MERCH][0]
user_rk = parties[22]
make_advice(user_rk, pred_MERCH_type)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


'237'

In [ ]:
test_data = pd.read_csv('/content/drive/My Drive/hackathon_data/merchant_types_for_test.csv').drop(columns = ['Unnamed: 0']).dropna()
test_data.head()

,party_rk,transaction_amt_rur,1,286,330,291,381,335,5,297,328,336,380,388,378,284,226,274,313,229,309,367,283,285,230,275,296,346,447,343,371,491,342,323,311,374,401,9,322,243,...,211,467,51,47,218,222,87,127,180,81,129,32,390,73,64,72,110,144,147,294,405,97,21,416,417,163,167,170,128,40,52,172,gender_cd,age,marital_status_desc,children_cnt,region_flg,user_transactions_by_month,n_prods,user_transactions_cat
0,1,62172.00,0.895580,0.000000,0.000000,0.000000,0.104420,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,55.0,2,1,0,16502.124545,2,1.0
1,5,111604.54,0.000000,0.110157,0.032983,0.001496,0.128122,0.000000,0.045482,0.049321,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.015483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,45.0,2,1,0,110706.375833,2,5.0
2,7,1275490.09,0.501533,0.000387,0.000000,0.001544,0.000000,0.003637,0.000000,0.005897,0.0,0.0,0.486636,0.000000,0.000193,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,45.0,2,4,0,376685.954444,1,5.0
3,8,5640.00,0.543972,0.082624,0.373404,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,60.0,2,1,0,19532.428571,2,1.0
4,9,3329.72,0.282907,0.090314,0.275699,0.000000,0.000000,0.000000,0.035739,0.000000,0.0,0.0,0.000000,0.315342,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,40.0,2,1,0,4319.370000,2,1.0


In [ ]:
len(parties), len(data.party_rk.values)

(8986, 45195)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

count = 0
for i in range(1,len(preds)):

    pred_MERCH_type = [p for p in preds[i] if p not in DROP_MERCH][0]
    user_rk = parties[i]
    if user_rk in data.party_rk.values:
        user_rk_num = data[data['party_rk'] == user_rk].index[0]
        advice = make_advice(user_rk_num, pred_MERCH_type)

        idx = test_data[test_data['party_rk'] == 7].index[0]
        user_test_merches = test_data.loc[idx][2:-9]
        user_test_merches = user_test_merches[user_test_merches > 0].index
        
        count += (advice in user_test_merches)


In [ ]:
# Count Users visited our recommended places
count/i

0.31018227529855436